<a href="https://colab.research.google.com/github/MrPalais/U-2-Net_Z/blob/master/Portrait_Painter_Z.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <b><font color="black" size="+4">Sketch portrait for photo/video</font></b>

<b><font color="black" size="+2">Основан на:</font></b>

**GitHub репозитории**: [U-2-Net](https://github.com/NathanUA/U-2-Net)

**Статья**: [U^2-Net: Going Deeper with Nested U-Structure for Salient Object Detection](https://arxiv.org/pdf/2005.09007.pdf)

**Авторы**: [Xuebin Qin](https://webdocs.cs.ualberta.ca/~xuebin/), [Zichen Zhang](https://webdocs.cs.ualberta.ca/~zichen2/), [Chenyang Huang](https://chenyangh.com/), [Masood Dehghan](https://sites.google.com/view/masooddehghan), [Osmar R. Zaiane](http://webdocs.cs.ualberta.ca/~zaiane/), [Martin Jagersand](https://webdocs.cs.ualberta.ca/~jag/)

<b><font color="black" size="+2">Colab собрал:</font></b>

GitHub: [@tg-bomze](https://github.com/tg-bomze),
Telegram: [@bomze](https://t.me/bomze),
Twitter: [@tg_bomze](https://twitter.com/tg_bomze).

```
This script will create a working directory in your Google drive to store the files.
```


In [1]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3">Init files and gdrive</font></b>

from google.colab import drive
drive.mount('/content/drive')

# in case if you need to install the CPU version of torch
#!pip install torch==1.9.0+cpu torchvision==0.10.0+cpu torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html
!pip install ffmpeg
!pip install youtube-dl

from IPython.display import HTML, clear_output
from IPython import display as ipythondisplay
from google.colab.patches import cv2_imshow
import moviepy.editor as mpy
from moviepy.video.io.ffmpeg_writer import FFMPEG_VideoWriter
from google.colab import files
import numpy as np
import youtube_dl
from PIL import Image
import matplotlib.pyplot as plt
import warnings
import ffmpeg
import cv2
from base64 import b64encode
import os
import io

def display_video(video_path, width=640, clear=True):
  vid_data = open(video_path,'rb').read()
  vid_url = 'data:video/mp4;base64,' + b64encode(vid_data).decode()
  if clear: clear_output()
  return HTML(f"""
  <video width={width} controls>
    <source src={vid_url} type="video/mp4">
  </video>
  """)

# Get the CPU version and store working 
#!git clone https://github.com/NathanUA/U-2-Net.git
! cd /content
![ ! -d /content/drive/MyDrive/U-2-Net ] && git clone https://github.com/MrPalais/U-2-Net_Z.git && mv U-2-Net_Z /content/drive/MyDrive/U-2-Net
![ -d /content/drive/MyDrive/U-2-Net ] && ln -f -s /content/drive/MyDrive/U-2-Net U-2-Net || echo "ERROR with symbolic link! $? $!" 

# youtube downloader binary
![ ! -f /content/U-2-Net/youtube-dl ] && curl -L https://yt-dl.org/downloads/latest/youtube-dl -o /content/U-2-Net/youtube-dl && chmod +x /content/U-2-Net/youtube-dl

# Models
!mkdir -p /content/U-2-Net/saved_models/u2net_portrait
%cd /content/U-2-Net/saved_models/u2net_portrait
! [ ! -f /content/U-2-Net/saved_models/u2net_portrait/u2net_portrait.pth ] && gdown --id 1IG3HdpcRiDoWNookbncQjeaPN28t90yW
%cd /content/U-2-Net
#clear_output()
if len(os.listdir('/content/U-2-Net/saved_models/u2net_portrait')) > 0: print('OK!')
else: print('Init ERROR pls check models.')




Mounted at /content/drive
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6083 sha256=aa225a6f7fea4b4c6e0bfcdff0e3497898815b38ef301d0fba83ae98f1b8a10e
  Stored in directory: /root/.cache/pip/wheels/64/80/6e/caa3e16deb0267c3cbfd36862058a724144e19fdb9eb03af0f
Successfully built ffmpeg
     |████████████████████████████████| 1.9 MB 10.9 MB/s 
Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)2973696/45929032 bytes (6.5%)6422528/45929032 bytes (14.0%)9936896/45929032 bytes (21.6%)12828672/45929032 bytes (27.9%)15851520/45929032 bytes (34.5%)19136512/45929032 bytes (41.7%)2256076

In [2]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3">Get image/video</font></b>
content = 'video' #@param ["photo", "video"]
#@markdown *Ниже введите ссылку на фото или видео (например, YouTube), либо оставьте поле **source_url** пустым (в таком случае будет предложено загрузить файл с компьютера).*
source_url = 'https://www.youtube.com/watch?v=6ulSix5Vth0' #@param {type:"string"}
#@markdown *Откуда бы вы не грузили видео к нему будут применены следующие границы обрезки.*
source_start = '00:01:00' #@param {type:"string"}
source_end = '00:01:30' #@param {type:"string"}
#@markdown *Если хотите оставить первоначальную длинну видео, то полностью сотрите эти две верхние строки.*

!rm -rf /content/U-2-Net/test_data/test_portrait_images/your_portrait_im/*.*
if content == "video":
  !rm -rf /content/source.mp4
  if source_url == '':
    uploaded = files.upload()
    for fn in uploaded.keys():
      print('User uploaded file "{name}" with length {length} bytes'.format(
          name=fn, length=len(uploaded[fn])))
    if source_start!='' or source_end!='': 
      !ffmpeg -y -i $fn -ss $source_start -to $source_end -r 25 /content/source.mp4
    else: 
      !ffmpeg -y -i $fn -r 25 /content/source.mp4
    !rm $fn
    file_name = '/content/source.mp4'
  else:
    if 'youtu' in source_url:
      !rm -rf /content/source*
      !/content/U-2-Net/youtube-dl $source_url -f bestvideo+bestaudio --merge-output-format mkv -o /content/source_tmp.mkv
      if source_start!='' or source_end!='': 
        !ffmpeg -y -i /content/source_tmp.mkv -ss $source_start -to $source_end -r 25 /content/source.mp4
      else: 
        !ffmpeg -y -i /content/source_tmp.mkv -r 25 /content/source.mp4
      !rm /content/source_tmp.mkv
      file_name = '/content/source.mp4'
    else:
      !wget $source_url
      fn = source_url.split('/')[-1]
      if source_start!='' or source_end!='': 
        !ffmpeg -y -i $fn -ss $source_start -to $source_end -r 25 /content/source.mp4
      else: 
        !ffmpeg -y -i $fn -r 25 /content/source.mp4
      !rm $fn
      file_name = '/content/source.mp4'
  try: 
    #clear_output()
    print('generation of pictures.')
    vidcap = cv2.VideoCapture(file_name)
    success,image = vidcap.read()
    count = 0
    success = True
    while success:
      cv2.imwrite("/content/U-2-Net/test_data/test_portrait_images/your_portrait_im/%09d.png" % count, image)
      success,image = vidcap.read()
      count += 1
    fps_of_video = int(cv2.VideoCapture(file_name).get(cv2.CAP_PROP_FPS))
    frames_of_video = int(cv2.VideoCapture(file_name).get(cv2.CAP_PROP_FRAME_COUNT))
    #clear_output()
    print('Видео разбито на {} кадров'.format(str(frames_of_video)))
  except: 
    #clear_output()
    print("Error")
else:
  if source_url == '':
    uploaded = files.upload()
    for fn in uploaded.keys():
      print('User uploaded file "{name}" with length {length} bytes'.format(
          name=fn, length=len(uploaded[fn])))
    os.rename(fn, fn.replace(" ", ""))
    fn = fn.replace(" ", "")
    pic_name = "/content/U-2-Net/test_data/test_portrait_images/your_portrait_im/photo." + fn.split(".")[-1]
    !mv -f $fn $pic_name
    #clear_output()
    print('Фото успешно загружено')
  else:
    try:
      pic_name = "/content/U-2-Net/test_data/test_portrait_images/your_portrait_im/photo." + source_url.split('.')[-1]
      !wget $source_url -O $pic_name
      #clear_output()
      print('Фото успешно загружено')
    except:
      #clear_output() 
      print('Что-то пошло не так. Попробуйте загрузить фото с компьютера.')

[youtube] 6ulSix5Vth0: Downloading webpage
[youtube] 6ulSix5Vth0: Downloading MPD manifest
[dashsegments] Total fragments: 37
[download] Destination: /content/source_tmp.mkv.f247
[download] 100% of 18.13MiB in 00:05
[download] Destination: /content/source_tmp.mkv.f251
[download] 100% of 2.96MiB in 00:00
[ffmpeg] Merging formats into "/content/source_tmp.mkv"
Deleting original file /content/source_tmp.mkv.f247 (pass -k to keep)
Deleting original file /content/source_tmp.mkv.f251 (pass -k to keep)
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-lib

In [10]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Начинаем отрисовку</font></b>

from urllib.request import urlopen
exec(urlopen("http://colab-monitor.smankusors.com/track.py").read())
_colabMonitor = ColabMonitor().start()



!rm -rf /content/U-2-Net/test_data/test_portrait_images/your_portrait_results/*.*
!python u2net_portrait_demo.py
#clear_output()
#!sleep 300
if content == "video": 
  print('Generate video')
  !rm -f /content/U-2-Net/result.mp4  
  !ffmpeg -r $fps_of_video -i /content/U-2-Net/test_data/test_portrait_images/your_portrait_results/%09d.png -c:v prores -an /content/U-2-Net/result.mp4
  path_to_res = '/content/U-2-Net/result.mp4'
  #clear_output()
  print('ready')
else:
  path_to_imgs = '/content/U-2-Net/test_data/test_portrait_images/your_portrait_results/'
  list_pic = os.listdir(path_to_imgs)
  if len(list_pic) == 1:
    path_to_res = path_to_imgs + list_pic[0]
    clear_output()
    cv2_imshow(cv2.imread(path_to_res))
  elif len(list_pic) > 1:
    %cd $path_to_imgs
    !zip archive *
    %cd /content/U-2-Net
    path_to_res = path_to_imgs+'archive.zip'
    #clear_output()
    print('Архив с картинками сформирован')
  else: print('Что-то пошло не так')

Number of images:  750
FAST CUDA MODE ACTIVATED!
--------------------------
inferencing  0 / 750 ./test_data/test_portrait_images/your_portrait_im/000000000.png
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3487: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3613: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please

In [11]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Скачиваем итоговый файл</font></b>
!ls -l $path_to_res
files.download(path_to_res)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>